# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Best Place to Start a new Firm

## Table of contents
* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction <a name="introduction"></a>

This notebook details how one can use the neighborhood data along with foursquare API to get the most common venues in a location thereby suggesting people where one can start a new business. I took the example of starting a new restaurant in the city of Toronto.

## Data <a name="data"></a>

I took the data available in the wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.
I scrapped the webpage using the beautiful soup to get the required data consisting of neighbourhoods, Boroughs and the respective postal codes. I then used the famous Foursquare API to explore neighborhoods in Toronto.

The next task is to predict the place where one can establish the new business. For this, I clustered the most common venue categories in each neighborhood using K means algorithm.

### Importing the BeautifulSoup to Scrap the webpage

In [2]:
from bs4 import BeautifulSoup
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#### Loading the html into soup

In [3]:
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())# Printing the page in html

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":890001695,"wgRevisionId":890001695,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

#### Retreiving the table tag

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">\n<tbody><tr>\n<th>Postcode</th>\n<th>Borough</th>\n<th>Neighbourhood\n</th></tr>\n<tr>\n<td>M1A</td>\n<td>Not assigned</td>\n<td>Not assigned\n</td></tr>\n<tr>\n<td>M2A</td>\n<td>Not assigned</td>\n<td>Not assigned\n</td></tr>\n<tr>\n<td>M3A</td>\n<td><a href="/wiki/North_York" title="North York">North York</a></td>\n<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>\n</td></tr>\n<tr>\n<td>M4A</td>\n<td><a href="/wiki/North_York" title="North York">North York</a></td>\n<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>\n</td></tr>\n<tr>\n<td>M5A</td>\n<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>\n<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>\n</td></tr>\n<tr>\n<td>M5A</td>\n<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>\n<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Par

#### Retreiving the td Tag

In [5]:
PostCode_td=My_table.findAll('td')
PostCode_td
#print(type(PostCode_a))

[<td>M1A</td>,
 <td>Not assigned</td>,
 <td>Not assigned\n</td>,
 <td>M2A</td>,
 <td>Not assigned</td>,
 <td>Not assigned\n</td>,
 <td>M3A</td>,
 <td><a href="/wiki/North_York" title="North York">North York</a></td>,
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>\n</td>,
 <td>M4A</td>,
 <td><a href="/wiki/North_York" title="North York">North York</a></td>,
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>\n</td>,
 <td>M5A</td>,
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>,
 <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>\n</td>,
 <td>M5A</td>,
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>,
 <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>\n</td>,
 <td>M6A</td>,
 <td><a href="/wiki/North_York" title="North York">North York</a></td>,
 <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heigh

#### Stripping the td Tag

In [6]:
PostCode_td[1].text.strip()
i=0
PostCode_td_le2=[]
while i <len(PostCode_td):
    PostCode_td_le2.append(PostCode_td[i].text.strip())# Using strip() function to trim the td Tag
    i=i+1
print(PostCode_td_le2)

[u'M1A', u'Not assigned', u'Not assigned', u'M2A', u'Not assigned', u'Not assigned', u'M3A', u'North York', u'Parkwoods', u'M4A', u'North York', u'Victoria Village', u'M5A', u'Downtown Toronto', u'Harbourfront', u'M5A', u'Downtown Toronto', u'Regent Park', u'M6A', u'North York', u'Lawrence Heights', u'M6A', u'North York', u'Lawrence Manor', u'M7A', u"Queen's Park", u'Not assigned', u'M8A', u'Not assigned', u'Not assigned', u'M9A', u'Etobicoke', u'Islington Avenue', u'M1B', u'Scarborough', u'Rouge', u'M1B', u'Scarborough', u'Malvern', u'M2B', u'Not assigned', u'Not assigned', u'M3B', u'North York', u'Don Mills North', u'M4B', u'East York', u'Woodbine Gardens', u'M4B', u'East York', u'Parkview Hill', u'M5B', u'Downtown Toronto', u'Ryerson', u'M5B', u'Downtown Toronto', u'Garden District', u'M6B', u'North York', u'Glencairn', u'M7B', u'Not assigned', u'Not assigned', u'M8B', u'Not assigned', u'Not assigned', u'M9B', u'Etobicoke', u'Cloverdale', u'M9B', u'Etobicoke', u'Islington', u'M9B', 

#### Getting only the Post Codes using the step size as 3

In [7]:
Postcode=PostCode_td_le2[0:len(PostCode_td):3] 
print(Postcode)

[u'M1A', u'M2A', u'M3A', u'M4A', u'M5A', u'M5A', u'M6A', u'M6A', u'M7A', u'M8A', u'M9A', u'M1B', u'M1B', u'M2B', u'M3B', u'M4B', u'M4B', u'M5B', u'M5B', u'M6B', u'M7B', u'M8B', u'M9B', u'M9B', u'M9B', u'M9B', u'M9B', u'M1C', u'M1C', u'M1C', u'M2C', u'M3C', u'M3C', u'M4C', u'M5C', u'M6C', u'M7C', u'M8C', u'M9C', u'M9C', u'M9C', u'M9C', u'M1E', u'M1E', u'M1E', u'M2E', u'M3E', u'M4E', u'M5E', u'M6E', u'M7E', u'M8E', u'M9E', u'M1G', u'M2G', u'M3G', u'M4G', u'M5G', u'M6G', u'M7G', u'M8G', u'M9G', u'M1H', u'M2H', u'M3H', u'M3H', u'M3H', u'M4H', u'M5H', u'M5H', u'M5H', u'M6H', u'M6H', u'M7H', u'M8H', u'M9H', u'M1J', u'M2J', u'M2J', u'M2J', u'M3J', u'M3J', u'M4J', u'M5J', u'M5J', u'M5J', u'M6J', u'M6J', u'M7J', u'M8J', u'M9J', u'M1K', u'M1K', u'M1K', u'M2K', u'M3K', u'M3K', u'M4K', u'M4K', u'M5K', u'M5K', u'M6K', u'M6K', u'M6K', u'M7K', u'M8K', u'M9K', u'M1L', u'M1L', u'M1L', u'M2L', u'M2L', u'M3L', u'M4L', u'M4L', u'M5L', u'M5L', u'M6L', u'M6L', u'M6L', u'M7L', u'M8L', u'M9L', u'M1M', u'M1M',

#### Defining a dataframe and inserting the Post code column 

In [8]:
import pandas as pd
df = pd.DataFrame()
df['Postcode'] = Postcode

df

,Postcode
0,M1A
1,M2A
2,M3A
3,M4A
4,M5A
5,M5A
6,M6A
7,M6A
8,M7A
9,M8A


#### Retreiving the Borough using the step size as 3

In [9]:
Borough=PostCode_td_le2[1:len(PostCode_td):3] 
print(Borough)

[u'Not assigned', u'Not assigned', u'North York', u'North York', u'Downtown Toronto', u'Downtown Toronto', u'North York', u'North York', u"Queen's Park", u'Not assigned', u'Etobicoke', u'Scarborough', u'Scarborough', u'Not assigned', u'North York', u'East York', u'East York', u'Downtown Toronto', u'Downtown Toronto', u'North York', u'Not assigned', u'Not assigned', u'Etobicoke', u'Etobicoke', u'Etobicoke', u'Etobicoke', u'Etobicoke', u'Scarborough', u'Scarborough', u'Scarborough', u'Not assigned', u'North York', u'North York', u'East York', u'Downtown Toronto', u'York', u'Not assigned', u'Not assigned', u'Etobicoke', u'Etobicoke', u'Etobicoke', u'Etobicoke', u'Scarborough', u'Scarborough', u'Scarborough', u'Not assigned', u'Not assigned', u'East Toronto', u'Downtown Toronto', u'York', u'Not assigned', u'Not assigned', u'Not assigned', u'Scarborough', u'Not assigned', u'Not assigned', u'East York', u'Downtown Toronto', u'Downtown Toronto', u'Not assigned', u'Not assigned', u'Not assigne

#### Inserting the Borough to Data frame

In [10]:
df['Borough'] = Borough
df

,Postcode,Borough
0,M1A,Not assigned
1,M2A,Not assigned
2,M3A,North York
3,M4A,North York
4,M5A,Downtown Toronto
5,M5A,Downtown Toronto
6,M6A,North York
7,M6A,North York
8,M7A,Queen's Park
9,M8A,Not assigned


#### Retreiving the Neighbourhood using the step size as 3

In [11]:
Neighbourhood=PostCode_td_le2[2:len(PostCode_td):3] 
print(Neighbourhood)

[u'Not assigned', u'Not assigned', u'Parkwoods', u'Victoria Village', u'Harbourfront', u'Regent Park', u'Lawrence Heights', u'Lawrence Manor', u'Not assigned', u'Not assigned', u'Islington Avenue', u'Rouge', u'Malvern', u'Not assigned', u'Don Mills North', u'Woodbine Gardens', u'Parkview Hill', u'Ryerson', u'Garden District', u'Glencairn', u'Not assigned', u'Not assigned', u'Cloverdale', u'Islington', u'Martin Grove', u'Princess Gardens', u'West Deane Park', u'Highland Creek', u'Rouge Hill', u'Port Union', u'Not assigned', u'Flemingdon Park', u'Don Mills South', u'Woodbine Heights', u'St. James Town', u'Humewood-Cedarvale', u'Not assigned', u'Not assigned', u'Bloordale Gardens', u'Eringate', u'Markland Wood', u'Old Burnhamthorpe', u'Guildwood', u'Morningside', u'West Hill', u'Not assigned', u'Not assigned', u'The Beaches', u'Berczy Park', u'Caledonia-Fairbanks', u'Not assigned', u'Not assigned', u'Not assigned', u'Woburn', u'Not assigned', u'Not assigned', u'Leaside', u'Central Bay Str

#### Inserting the Neighbourhood to DataFrame

In [12]:
df['Neighbourhood'] = Neighbourhood
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


#### Dropping the rows which has Not assigned in Borough column

In [13]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Joining the Neighbourhoods belonging to same postcode

In [14]:
df= df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [15]:
df.loc[df['Postcode'] == 'M7A', 'Neighbourhood'] = "Queen's Park"
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


#### Importing the Geospatial Data

In [16]:
import pandas as pd
df2=pd.read_csv('http://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df2.rename(columns={'Postal Code':'Postcode'},inplace=True)
df2.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging the two dataframes to combine the Latitude and Longitude to the Original DataFrame

In [18]:
Merged_DF=pd.merge(df,df2,on='Postcode',how='inner')
Merged_DF.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
Merged_DF.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Segmenting and Clustering

In [19]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Getting the coordinates of Toronto City

In [20]:
address = 'Toronto City'

geolocator = Nominatim(user_agent="tt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.7394839, -79.369314.


#### Creating a folium Map to visulaize the city

In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Merged_DF['Latitude'], Merged_DF['Longitude'], Merged_DF['Borough'], Merged_DF['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [22]:
# @hidden_cell
CLIENT_ID = 'EPSD3PU5MEKRWQJ33TVM3535GHG11GAVHIQEBKGYIVEMOMZR' # your Foursquare ID
CLIENT_SECRET = 'FFCQ2ERCSVLUTZOSATJJL5Q5FXDS10RC3OA2ZRSFGHZN03PO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


#### First neighborhood in our dataframe.

In [23]:
Merged_DF.loc[0, 'Neighborhood']

u'Rouge,Malvern'

First neighborhood's latitude and longitude values.

In [24]:
neighborhood_latitude = Merged_DF.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Merged_DF.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Merged_DF.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.8066863, -79.1943534.


#### Getting the top 100 venues that are in Rouge Malvern within a radius of 500 meters.

In [25]:
LIMIT = 100
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=EPSD3PU5MEKRWQJ33TVM3535GHG11GAVHIQEBKGYIVEMOMZR&client_secret=FFCQ2ERCSVLUTZOSATJJL5Q5FXDS10RC3OA2ZRSFGHZN03PO&v=20180605&ll=43.8066863,-79.1943534&radius=500&limit=100'

In [26]:
results = requests.get(url).json()
results

{u'meta': {u'code': 200, u'requestId': u'5c9f176cf594df6af977bef1'},
 u'response': {u'groups': [{u'items': [{u'reasons': {u'count': 0,
       u'items': [{u'reasonName': u'globalInteractionReason',
         u'summary': u'This spot is popular',
         u'type': u'general'}]},
      u'referralId': u'e-0-4bb6b9446edc76b0d771311c-0',
      u'venue': {u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          u'suffix': u'.png'},
         u'id': u'4bf58dd8d48988d16e941735',
         u'name': u'Fast Food Restaurant',
         u'pluralName': u'Fast Food Restaurants',
         u'primary': True,
         u'shortName': u'Fast Food'}],
       u'id': u'4bb6b9446edc76b0d771311c',
       u'location': {u'cc': u'CA',
        u'city': u'Toronto',
        u'country': u'Canada',
        u'crossStreet': u'Morningside & Sheppard',
        u'distance': 387,
        u'formattedAddress': [u'Toronto ON', u'Canada'],
        u'labeledLatLngs': [{u'label': u'display

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


### Explore Neighborhoods in Toronto within 500 radius

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
toronto_venues = getNearbyVenues(names=Merged_DF['Neighborhood'],
                                   latitudes=Merged_DF['Latitude'],
                                   longitudes=Merged_DF['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [32]:
print(Merged_DF.shape)
Merged_DF.head()

(103, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",24,24,24,24,24,24
Berczy Park,55,55,55,55,55,55


In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


## Methodology <a name="methodology"></a>

The main focus of this project is to identify the suitable place in which one can start a new restaurant in the city of Toronto.

The steps included in order to achieve our task are listed below.

Task 1: Retrieve the data of the Toronto city venues using the Foursquare API.

Task 2: Combine the venues of each neighbourhood using K Means clustering algorithm.

Task 3: Get the top most venues of each location.

Task 4: Take the least venue where there are less number of restaurants available, so that one can get more profits because of         less competition.

Task 5: The obtained Results can then be used to identify the suitable location for staring a new restaurant

## Analysis <a name="analysis"></a>

In [35]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [36]:
toronto_onehot.shape

(2235, 273)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.01,0.000000,0.000

In [38]:
toronto_grouped.shape

(98, 273)

#### Printing each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.06
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Agincourt----
            venue  freq
0    Skating Rink  0.25
1  Breakfast Spot  0.25
2  Sandwich Place  0.25
3          Lounge  0.25
4          Market  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2         Mediterranean Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store   0.2
1              Pharmacy   0.1
2           Coffee Shop   0.1
3  Fast Food Restaurant   0.1
4   Fried Chicken Joint   0.1


----Alderwood,Long Branch----
            venue  f

           venue  freq
0  Shopping Mall  0.25
1           Bank  0.25
2  Grocery Store  0.25
3          Hotel  0.25
4    Yoga Studio  0.00


----Downsview,North Park,Upwood Park----
                        venue  freq
0            Basketball Court  0.25
1  Construction & Landscaping  0.25
2                      Bakery  0.25
3                        Park  0.25
4                 Men's Store  0.00


----East Birchmount Park,Ionview,Kennedy Park----
               venue  freq
0     Discount Store  0.17
1  Convenience Store  0.17
2        Bus Station  0.17
3   Department Store  0.17
4        Coffee Shop  0.17


----East Toronto----
                venue  freq
0                Park  0.50
1         Coffee Shop  0.25
2   Convenience Store  0.25
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Emery,Humberlea----
                             venue  freq
0                   Baseball Field   0.5
1    Paper / Office Supplies Store   0.5
2                      Yoga Studio   0.0
3  Molec

              venue  freq
0       Pizza Place  0.08
1       Coffee Shop  0.08
2              Café  0.08
3  Sushi Restaurant  0.05
4          Tea Room  0.05


----Ryerson,Garden District----
                       venue  freq
0                Coffee Shop  0.08
1             Clothing Store  0.06
2                       Café  0.04
3             Cosmetics Shop  0.04
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                             venue  freq
0                       Playground   1.0
1                      Men's Store   0.0
2              Monument / Landmark   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.06
2           Hotel  0.05
3            Café  0.05
4  Clothing Store  0.04


----Stn A PO Boxes 25 The Esplanade----
         venue  freq
0  Coffee Shop  0.10
1   Restaurant  0.04
2         Café  0.04
3          Pub  0.03
4       

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Creating a new dataframe and display the top 10 venues for each neighborhood.

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Bar,Thai Restaurant,Steakhouse,Café,American Restaurant,Bakery,Hotel,Sushi Restaurant,Asian Restaurant
1,Agincourt,Lounge,Sandwich Place,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Coffee Shop,Beer Store,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Liquor Store,Pizza Place,Dim Sum Restaurant
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Coffee Shop,Gym,Skating Rink,Sandwich Place,Dance Studio,Pool,Pub,Diner


### Clustering Neighborhoods using K Means algorithm

In [42]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Merged_DF

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Wings Joint
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3.0,Bar,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Medical Center,Electronics Store,Pizza Place,Spa,Mexican Restaurant,Rental Car Location,Breakfast Spot,Intersection,Women's Store,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Athletics & Sports,Bank,Bakery,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the Clusters

#### Cluster 1

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1stMostCommonVenue,2ndMostCommonVenue,3rdMostCommonVenue,4thMostCommonVenue,5thMostCommonVenue,6thMostCommonVenue,7thMostCommonVenue,8thMostCommonVenue,9thMostCommonVenue,10thMostCommonVenue
0,Scarborough,0.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Wings Joint
2,Scarborough,0.0,Medical Center,Electronics Store,Pizza Place,Spa,Mexican Restaurant,Rental Car Location,Breakfast Spot,Intersection,Women's Store,Diner
3,Scarborough,0.0,Coffee Shop,Korean Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,Scarborough,0.0,Athletics & Sports,Bank,Bakery,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
6,Scarborough,0.0,Department Store,Convenience Store,Discount Store,Chinese Restaurant,Bus Station,Coffee Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
7,Scarborough,0.0,Bakery,Bus Line,Park,Bus Station,Fast Food Restaurant,Intersection,Soccer Field,Convenience Store,Cosmetics Shop,Event Space
8,Scarborough,0.0,Motel,Movie Theater,American Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,Scarborough,0.0,Café,College Stadium,Skating Rink,General Entertainment,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,Scarborough,0.0,Indian Restaurant,Pet Store,Latin American Restaurant,Vietnamese Restaurant,Furniture / Home Store,Chinese Restaurant,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant
11,Scarborough,0.0,Middle Eastern Restaurant,Breakfast Spot,Sandwich Place,Shopping Mall,Auto Garage,Smoke Shop,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Event Space


#### Cluster 2

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1stMostCommonVenue,2ndMostCommonVenue,3rdMostCommonVenue,4thMostCommonVenue,5thMostCommonVenue,6thMostCommonVenue,7thMostCommonVenue,8thMostCommonVenue,9thMostCommonVenue,10thMostCommonVenue
5,Scarborough,1.0,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
14,Scarborough,1.0,Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
48,Central Toronto,1.0,Gym,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


#### Cluster 3

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1stMostCommonVenue,2ndMostCommonVenue,3rdMostCommonVenue,4thMostCommonVenue,5thMostCommonVenue,6thMostCommonVenue,7thMostCommonVenue,8thMostCommonVenue,9thMostCommonVenue,10thMostCommonVenue
23,North York,2.0,Park,Bank,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
25,North York,2.0,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
30,North York,2.0,Bus Stop,Park,Airport,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
40,East York,2.0,Park,Convenience Store,Coffee Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,2.0,Gym / Fitness Center,Park,Bus Line,Swim School,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
50,Downtown Toronto,2.0,Park,Playground,Trail,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
74,York,2.0,Park,Women's Store,Market,Fast Food Restaurant,Pharmacy,Comfort Food Restaurant,Comic Shop,Ethiopian Restaurant,College Stadium,Empanada Restaurant
79,North York,2.0,Park,Bakery,Construction & Landscaping,Basketball Court,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
90,Etobicoke,2.0,Park,Pool,Smoke Shop,River,Colombian Restaurant,Department Store,Empanada Restaurant,Electronics Store,College Rec Center,Eastern European Restaurant
100,Etobicoke,2.0,Pizza Place,Park,Bus Line,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


#### Cluster 4

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1stMostCommonVenue,2ndMostCommonVenue,3rdMostCommonVenue,4thMostCommonVenue,5thMostCommonVenue,6thMostCommonVenue,7thMostCommonVenue,8thMostCommonVenue,9thMostCommonVenue,10thMostCommonVenue
1,Scarborough,3.0,Bar,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Fast Food Restaurant


#### Cluster 5

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1stMostCommonVenue,2ndMostCommonVenue,3rdMostCommonVenue,4thMostCommonVenue,5thMostCommonVenue,6thMostCommonVenue,7thMostCommonVenue,8thMostCommonVenue,9thMostCommonVenue,10thMostCommonVenue
91,Etobicoke,4.0,Baseball Field,Pool,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
97,North York,4.0,Baseball Field,Paper / Office Supplies Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store


In [45]:
toronto_merged.head(100)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Wings Joint
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3.0,Bar,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Medical Center,Electronics Store,Pizza Place,Spa,Mexican Restaurant,Rental Car Location,Breakfast Spot,Intersection,Women's Store,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Athletics & Sports,Bank,Bakery,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1.0,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0.0,Department Store,Convenience Store,Discount Store,Chinese Restaurant,Bus Station,Coffee Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0.0,Bakery,Bus Line,Park,Bus Station,Fast Food Restaurant,Intersection,Soccer Field,Convenience Store,Cosmetics Shop,Event Space
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0.0,Motel,Movie Theater,American Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0.0,Café,College Stadium,Skating Rink,General Entertainment,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


Renaming the columns

In [46]:
toronto_merged.rename(columns={'1st Most Common Venue':'1stMostCommonVenue'},inplace=True)
toronto_merged.rename(columns={'2nd Most Common Venue':'2ndMostCommonVenue'},inplace=True)
toronto_merged.rename(columns={'3rd Most Common Venue':'3rdMostCommonVenue'},inplace=True)
toronto_merged.rename(columns={'4th Most Common Venue':'4thMostCommonVenue'},inplace=True)
toronto_merged.rename(columns={'5th Most Common Venue':'5thMostCommonVenue'},inplace=True)
toronto_merged.rename(columns={'6th Most Common Venue':'6thMostCommonVenue'},inplace=True)
toronto_merged.rename(columns={'7th Most Common Venue':'7thMostCommonVenue'},inplace=True)
toronto_merged.rename(columns={'8th Most Common Venue':'8thMostCommonVenue'},inplace=True)
toronto_merged.rename(columns={'9th Most Common Venue':'9thMostCommonVenue'},inplace=True)
toronto_merged.rename(columns={'10th Most Common Venue':'10thMostCommonVenue'},inplace=True)
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1stMostCommonVenue,2ndMostCommonVenue,3rdMostCommonVenue,4thMostCommonVenue,5thMostCommonVenue,6thMostCommonVenue,7thMostCommonVenue,8thMostCommonVenue,9thMostCommonVenue,10thMostCommonVenue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Wings Joint
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3.0,Bar,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Medical Center,Electronics Store,Pizza Place,Spa,Mexican Restaurant,Rental Car Location,Breakfast Spot,Intersection,Women's Store,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Athletics & Sports,Bank,Bakery,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Dropping the rows where there is no data

In [47]:
toronto_merged.dropna(inplace=True)

#### Getting the place where there are more number of restaurants. These places will not yield high profits as there are already established more number of restaurants

In [48]:
High_Restaurant_df=toronto_merged[toronto_merged['1stMostCommonVenue'].str.contains('Restaurant')]
# Taking the rows which has the venues like Restaurant

In [49]:
High_Restaurant_df

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1stMostCommonVenue,2ndMostCommonVenue,3rdMostCommonVenue,4thMostCommonVenue,5thMostCommonVenue,6thMostCommonVenue,7thMostCommonVenue,8thMostCommonVenue,9thMostCommonVenue,10thMostCommonVenue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Wings Joint
10,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford He...",43.757410,-79.273304,0.0,Indian Restaurant,Pet Store,Latin American Restaurant,Vietnamese Restaurant,Furniture / Home Store,Chinese Restaurant,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant
11,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849,0.0,Middle Eastern Restaurant,Breakfast Spot,Sandwich Place,Shopping Mall,Auto Garage,Smoke Shop,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Event Space
15,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,0.0,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Grocery Store,Pharmacy,Sandwich Place,Coffee Shop,Breakfast Spot,American Restaurant,Cosmetics Shop
22,M2N,North York,Willowdale South,43.770120,-79.408493,0.0,Restaurant,Ramen Restaurant,Sandwich Place,Shopping Mall,Café,Coffee Shop,Pizza Place,Fast Food Restaurant,Steakhouse,Ice Cream Shop
25,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
27,M3C,North York,"Flemingdon Park,Don Mills South",43.725900,-79.340923,0.0,Asian Restaurant,Gym,Coffee Shop,Beer Store,Grocery Store,Clothing Store,Chinese Restaurant,Restaurant,Dim Sum Restaurant,Discount Store
34,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Portuguese Restaurant,Coffee Shop,Intersection,Hockey Arena,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
35,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,0.0,Fast Food Restaurant,Pizza Place,Pharmacy,Athletics & Sports,Gastropub,Intersection,Pet Store,Café,Rock Climbing Spot,Breakfast Spot
39,M4H,East York,Thorncliffe Park,43.705369,-79.349372,0.0,Indian Restaurant,Yoga Studio,Supermarket,Gym,Housing Development,Liquor Store,Discount Store,Park,Pharmacy,Pizza Place


#### Getting the place where there are less number of restaurants. These places will yield high profits as there are less number of restaurants

In [50]:
Low_Restaurant_df=toronto_merged[toronto_merged['10thMostCommonVenue'].str.contains('Restaurant') | toronto_merged['10thMostCommonVenue'].str.contains('Diner')]

In [51]:
Low_Restaurant_df

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1stMostCommonVenue,2ndMostCommonVenue,3rdMostCommonVenue,4thMostCommonVenue,5thMostCommonVenue,6thMostCommonVenue,7thMostCommonVenue,8thMostCommonVenue,9thMostCommonVenue,10thMostCommonVenue
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3.0,Bar,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Medical Center,Electronics Store,Pizza Place,Spa,Mexican Restaurant,Rental Car Location,Breakfast Spot,Intersection,Women's Store,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Athletics & Sports,Bank,Bakery,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1.0,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0.0,Department Store,Convenience Store,Discount Store,Chinese Restaurant,Bus Station,Coffee Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0.0,Café,College Stadium,Skating Rink,General Entertainment,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford He...",43.757410,-79.273304,0.0,Indian Restaurant,Pet Store,Latin American Restaurant,Vietnamese Restaurant,Furniture / Home Store,Chinese Restaurant,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,0.0,Lounge,Sandwich Place,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
13,M1T,Scarborough,"Clarks Corners,Sullivan,Tam O'Shanter",43.781638,-79.304302,0.0,Pizza Place,Pharmacy,Noodle House,Fried Chicken Joint,Thai Restaurant,Italian Restaurant,Chinese Restaurant,Fast Food Restaurant,Drugstore,Dumpling Restaurant


#### Getting the places where there are less number of restaurants

In [52]:
Low_Resataurant_Places=Low_Restaurant_df['Borough'].unique()
Low_Resataurant_Places

array([u'Scarborough', u'North York', u'Central Toronto',
       u'Downtown Toronto', u'York', u'West Toronto', u'East Toronto',
       u'Etobicoke'], dtype=object)

In [53]:
Low_Resataurant_Places.astype('str')

array(['Scarborough', 'North York', 'Central Toronto', 'Downtown Toronto',
       'York', 'West Toronto', 'East Toronto', 'Etobicoke'], dtype='|S16')

In [54]:
import pandas as pd
Low_Resataurant_Places_DF=pd.DataFrame(Low_Resataurant_Places,columns=['Boroughs'])# Importing the results into a dataframe

## Results and Discussion <a name="results"></a>

After doing all our analysis on exploring the Toronto city, we got the places where there are less number of restaurants. These places can be used by the investors who are planning to start a new business in the city of Toronto thereby increasing their profits.

I used the less common places where the restaurants are available, so that one can use these places for starting the new restaurants.

In [55]:
Low_Resataurant_Places_DF

,Boroughs
0,Scarborough
1,North York
2,Central Toronto
3,Downtown Toronto
4,York
5,West Toronto
6,East Toronto
7,Etobicoke


## Conclusion <a name="conclusion"></a>

The main purpose of this project is to find the places where one can start their new business such as a restaurant. Not only the restaurant, but we can also predict the places such as spa, parks, Coffee shops and many other places. These can be further used to explore the city to find the prospective place for starting a new business.